imports & connection

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy import text
import pandas as pd
import sqlite3

DWengine = create_engine("mssql+pyodbc://LEGION\\SQLEXPRESS/Data Warehouse?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

Create dataframes from DB tables

In [ ]:
course_df = pd.read_sql_query('SELECT * FROM course', DWengine)
order_df = pd.read_sql_query('SELECT * FROM [order]', DWengine)
order_details_df = pd.read_sql_query('SELECT * FROM order_details', DWengine)
order_method_df = pd.read_sql_query('SELECT * FROM order_method', DWengine)
retailer_site_df = pd.read_sql_query('SELECT * FROM retailer_site', DWengine)
return_reason_df = pd.read_sql_query('SELECT * FROM return_reason', DWengine)
returned_item_df = pd.read_sql_query('SELECT * FROM returned_item', DWengine)
sales_staff_df = pd.read_sql_query('SELECT * FROM sales_Staff', DWengine)
training_df = pd.read_sql_query('SELECT * FROM training', DWengine)

Source Data Model connection

In [ ]:
SDMengine = create_engine("mssql+pyodbc://LEGION\\SQLEXPRESS/sdm?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

course table data insert

In [ ]:
course_sdm_df = pd.read_sql_query('SELECT * FROM course', SDMengine)

course_sdm_df.to_sql('course', con=DWengine, if_exists='append', index=False) 

return_reason table data insert

In [ ]:
return_reason_sdm_df = pd.read_sql_query('SELECT * FROM return_reason', SDMengine)

return_reason_sdm_df.to_sql('return_reason', con=DWengine, if_exists='append', index=False)

order_method table data insert

In [ ]:
order_method_sdm_df = pd.read_sql_query('SELECT * FROM order_method', SDMengine)

order_method_sdm_df.to_sql('order_method', con=DWengine, if_exists='append', index=False)

retailer_site table data insert

In [ ]:
retailer_site_sdm_df = pd.read_sql_query('SELECT * from retailer_site', SDMengine)
country_sdm_df = pd.read_sql_query('SELECT * FROM country', SDMengine)

merged_retailer_site_df = retailer_site_sdm_df.merge(country_sdm_df, on=['COUNTRY_CODE','COUNTRY_CODE'])

retailerSiteCounter = 0
while retailerSiteCounter < len(merged_retailer_site_df):
    retailer_site_temp_df = pd.DataFrame({'RETAILER_SITE_CODE': [merged_retailer_site_df.iloc[retailerSiteCounter]['RETAILER_SITE_CODE']], 'RETAILER_CODE': [merged_retailer_site_df.iloc[retailerSiteCounter]['RETAILER_CODE']], 'ADDRESS1': [merged_retailer_site_df.iloc[retailerSiteCounter]['ADDRESS1']], 'ADDRESS2': [merged_retailer_site_df.iloc[retailerSiteCounter]['ADDRESS2']], 'CITY': [merged_retailer_site_df.iloc[retailerSiteCounter]['CITY']], 'REGION': [merged_retailer_site_df.iloc[retailerSiteCounter]['REGION']], 'COUNTRY_CODE': [merged_retailer_site_df.iloc[retailerSiteCounter]['COUNTRY_CODE']], 'COUNTRY': [merged_retailer_site_df.iloc[retailerSiteCounter]['COUNTRY_EN']]}) 
    
    retailer_site_temp_df.to_sql('retailer_site', con=DWengine, if_exists='append', index=False)
    retailerSiteCounter += 1

sales_staff table data insert

In [ ]:
sales_staff_sdm_df = pd.read_sql_query('SELECT * FROM sales_staff', SDMengine)
sales_branch_sdm_df = pd.read_sql_query('SELECT * FROM sales_branch', SDMengine)
sales_territory_sdm_Df = pd.read_sql_query('SELECT * FROM sales_territory', SDMengine)

merged_sales_staff_df = country_sdm_df.merge(sales_territory_sdm_Df, on=['SALES_TERRITORY_CODE', 'SALES_TERRITORY_CODE'])
merged_sales_staff_df = merged_sales_staff_df.merge(sales_branch_sdm_df, on=['COUNTRY_CODE','COUNTRY_CODE'])
merged_sales_staff_df = merged_sales_staff_df.merge(sales_staff_sdm_df, on=['SALES_BRANCH_CODE', 'SALES_BRANCH_CODE'])

salesStaffCounter = 0
while salesStaffCounter < len(merged_sales_staff_df):
    sales_staff_temp_df = pd.DataFrame({
        'SALES_STAFF_CODE': [merged_sales_staff_df.iloc[salesStaffCounter]['SALES_STAFF_CODE']], 
        'FIRST_NAME': [merged_sales_staff_df.iloc[salesStaffCounter]['FIRST_NAME']], 
        'LAST_NAME': [merged_sales_staff_df.iloc[salesStaffCounter]['LAST_NAME']],
        'POSITION_EN': [merged_sales_staff_df.iloc[salesStaffCounter]['POSITION_EN']],
        'WORK_PHONE': [merged_sales_staff_df.iloc[salesStaffCounter]['WORK_PHONE']],
        'EXTENSION': [merged_sales_staff_df.iloc[salesStaffCounter]['EXTENSION']],
        'EMAIL': [merged_sales_staff_df.iloc[salesStaffCounter]['EMAIL']],
        'DATE_HIRED': [merged_sales_staff_df.iloc[salesStaffCounter]['DATE_HIRED']],
        'SALES_BRANCH_CODE': [merged_sales_staff_df.iloc[salesStaffCounter]['SALES_BRANCH_CODE']],
        'ADDRESS1': [merged_sales_staff_df.iloc[salesStaffCounter]['ADDRESS1']],
        'ADDRESS2': [merged_sales_staff_df.iloc[salesStaffCounter]['ADDRESS2']],
        'CITY': [merged_sales_staff_df.iloc[salesStaffCounter]['CITY']],
        'REGION': [merged_sales_staff_df.iloc[salesStaffCounter]['REGION']],
        'COUNTRY_CODE': [merged_sales_staff_df.iloc[salesStaffCounter]['COUNTRY_CODE']],
        'COUNTRY': [merged_sales_staff_df.iloc[salesStaffCounter]['COUNTRY_EN']],
        'LANGUAGE': [merged_sales_staff_df.iloc[salesStaffCounter]['LANGUAGE']],
        'TERRITORY_CODE': [merged_sales_staff_df.iloc[salesStaffCounter]['SALES_TERRITORY_CODE']],
        'TERRITORY_NAME': [merged_sales_staff_df.iloc[salesStaffCounter]['TERRITORY_NAME_EN']]})
    
    sales_staff_temp_df.to_sql('sales_staff', con=DWengine, if_exists='append', index=False)
    salesStaffCounter += 1

order table data insert

In [ ]:
order_sdm_df = pd.read_sql_query('SELECT * FROM order_header', SDMengine)

orderCounter = 0
while orderCounter < len(order_sdm_df):
    order_temp_df = pd.DataFrame({
        'ORDER_NUMBER': [order_sdm_df.iloc[orderCounter]['ORDER_NUMBER']], 
        'RETAILER_NAME': [order_sdm_df.iloc[orderCounter]['RETAILER_NAME']],
        'RETAILER_SITE_CODE': [order_sdm_df.iloc[orderCounter]['RETAILER_SITE_CODE']],
        'RETAILER_CONTACT_CODE': [order_sdm_df.iloc[orderCounter]['RETAILER_CONTACT_CODE']],
        'SALES_STAFF_CODE': [order_sdm_df.iloc[orderCounter]['SALES_STAFF_CODE']],
        'SALES_BRANCH_CODE': [order_sdm_df.iloc[orderCounter]['SALES_BRANCH_CODE']],
        'ORDER_DATE': [order_sdm_df.iloc[orderCounter]['ORDER_DATE']],
        'ORDER_METHOD_CODE': [order_sdm_df.iloc[orderCounter]['ORDER_METHOD_CODE']]})
    
    order_temp_df.to_sql('order', con=DWengine, if_exists='append', index=False)
    orderCounter += 1

order_details table data insert

In [ ]:
order_details_sdm_df = pd.read_sql_query('SELECT * FROM order_details', SDMengine)
product_sdm_df = pd.read_sql_query('SELECT * FROM product', SDMengine)
product_type_sdm_df = pd.read_sql_query('SELECT * FROM product_type', SDMengine)
product_line_sdm_df = pd.read_sql_query('SELECT * FROM product_line', SDMengine)

merged_order_details_df = product_type_sdm_df.merge(product_line_sdm_df, on=['PRODUCT_LINE_CODE', 'PRODUCT_LINE_CODE'])
merged_order_details_df = merged_order_details_df.merge(product_sdm_df, on=['PRODUCT_TYPE_CODE', 'PRODUCT_TYPE_CODE'])
merged_order_details_df = merged_order_details_df.merge(order_details_sdm_df, on=['PRODUCT_NUMBER', 'PRODUCT_NUMBER'])
merged_order_details_df = merged_order_details_df.merge(order_sdm_df, on=['ORDER_NUMBER', 'ORDER_NUMBER'])

orderDetailsCount = 0
while orderDetailsCount < len(merged_order_details_df):
    order_details_temp_df = pd.DataFrame({
        'ORDER_DETAIL_CODE': [merged_order_details_df.iloc[orderDetailsCount]['ORDER_DETAIL_CODE']],
        'ORDER_NUMBER': [merged_order_details_df.iloc[orderDetailsCount]['ORDER_NUMBER']],
        'PRODUCT_NUMBER': [merged_order_details_df.iloc[orderDetailsCount]['PRODUCT_NUMBER']],
        'QUANTITY': [merged_order_details_df.iloc[orderDetailsCount]['QUANTITY']],
        'UNIT_COST': [merged_order_details_df.iloc[orderDetailsCount]['UNIT_COST']],
        'UNIT_PRICE': [merged_order_details_df.iloc[orderDetailsCount]['UNIT_PRICE']],
        'UNIT_SALE_PRICE': [merged_order_details_df.iloc[orderDetailsCount]['UNIT_SALE_PRICE']],
        'PRODUCT_IMAGE': [merged_order_details_df.iloc[orderDetailsCount]['PRODUCT_IMAGE']],
        'PRODUCT_NAME': [merged_order_details_df.iloc[orderDetailsCount]['PRODUCT_NAME']],
        'DESCRIPTION': [merged_order_details_df.iloc[orderDetailsCount]['DESCRIPTION']],
        'PRODUCT_TYPE_CODE': [merged_order_details_df.iloc[orderDetailsCount]['PRODUCT_TYPE_CODE']],
        'PRODUCT_TYPE_EN': [merged_order_details_df.iloc[orderDetailsCount]['PRODUCT_TYPE_EN']],
        'PRODUCT_LINE_CODE': [merged_order_details_df.iloc[orderDetailsCount]['PRODUCT_LINE_CODE']],
        'PRODUCT_LINE_EN': [merged_order_details_df.iloc[orderDetailsCount]['PRODUCT_LINE_EN']],
        'RETAILER_NAME': [merged_order_details_df.iloc[orderDetailsCount]['RETAILER_NAME']],
        'ORDER_DATE': [merged_order_details_df.iloc[orderDetailsCount]['ORDER_DATE']],
    })

    order_details_temp_df.to_sql('order_details', con=DWengine, if_exists='append', index=False)
    orderDetailsCount += 1

returned_item table data insert

In [46]:
returned_item_sdm_df = pd.read_sql_query('SELECT * FROM returned_item', SDMengine)

returnedItemCounter = 0
while returnedItemCounter < len(returned_item_sdm_df):
    returned_item_temp_df = pd.DataFrame({
        'RETURN_CODE': [returned_item_sdm_df.iloc[returnedItemCounter]['RETURN_CODE']],
        'RETURN_DATE': [returned_item_sdm_df.iloc[returnedItemCounter]['RETURN_DATE']],
        'ORDER_DETAIL_CODE': [returned_item_sdm_df.iloc[returnedItemCounter]['ORDER_DETAIL_CODE']],
        'RETURN_REASON_CODE': [returned_item_sdm_df.iloc[returnedItemCounter]['RETURN_REASON_CODE']],
        'RETURN_QUANTITY': [returned_item_sdm_df.iloc[returnedItemCounter]['RETURN_QUANTITY']],
    })

    returned_item_temp_df.to_sql('returned_item', con=DWengine, if_exists='append', index=False)
    returnedItemCounter += 1

training table data insert

In [49]:
training_sdm_df = pd.read_sql_query('SELECT * FROM training', SDMengine)

trainingCounter = 0
while trainingCounter < len(training_sdm_df):
    training_temp_df = pd.DataFrame({
        'YEAR': [training_sdm_df.iloc[trainingCounter]['YEAR']],
        'SALES_STAFF_CODE': [training_sdm_df.iloc[trainingCounter]['SALES_STAFF_CODE']],
        'COURSE_CODE': [training_sdm_df.iloc[trainingCounter]['COURSE_CODE']],
    })

    training_temp_df.to_sql('training', con=DWengine, if_exists='append', index=False)
    trainingCounter += 1